In [1]:
%load_ext autoreload
%autoreload 2

## Running experiments externally

Running experiments externally is the most common use of summit. There is a simple set of commands that allows you to do this. Let's say we are going to run the SnarBenchmark ourselves instead of in a closed-loop setup. We'll use TSEMO since this is a multiobjective problem.

In [14]:
# Import the necessary classes
from summit.benchmarks import SnarBenchmark
from summit.strategies import TSEMO2
from summit.utils.dataset import DataSet

In [15]:
# We are just going to get the domain from the SnarBenchmark
exp = SnarBenchmark()

# Instatiate TSEMO
tsemo = TSEMO2(exp.domain)

# To get the first experiments
next_experiments = tsemo.suggest_experiments(5)
next_experiments

,tau,equiv_pldn,conc_dfnb,temperature,strategy
0,0.65,2.2,0.46,75.0,LHS
1,0.95,3.0,0.38,111.0,LHS
2,1.25,4.6,0.14,57.0,LHS
3,1.85,3.8,0.30,39.0,LHS
4,1.55,1.4,0.22,93.0,LHS


You'll notice that the first set of experiments was generated by LHS or latin hypercube sampling. We'll need to go to the lab and run experiments at those conditions. 

Since it might be a while before we come back, let's save what we've done up until now. We save a file called `snar_tsemo_external.json` which contains all of the information we need to reproduce our work up to this point.  We will also produce a CSV with the suggested conditions, which we can use to record the results of our experiments. 

In [16]:
# Save TSEMO hyperparameters and internal state
tsemo.save('assets/snar_tsemo_external.json')
# Save experiments to CSV
next_experiments.to_csv('assets/snar_experiments_external_0.csv')

Normally, we would now go and run the experiments separately and add a column for sty and yield with our results. We could do that in Excel. However, since this is just a demonstration, we'll simulate the experiments using the SnarBenchmark class and save the results to a CSV. 

In [17]:
# "Run" the experiments and save to CSV.
results = exp.run_experiments(next_experiments)
results.to_csv('assets/snar_experiments_external_1.csv')

Now that we have run the conditions, we can load TSEMO and the data back in from the files we saved.

In [24]:
# Load in previous TSEMO hyperparameters and internal state
tsemo = TSEMO2.load('assets/snar_tsemo_external.json')

# Load in experimental results
prev_res = DataSet.read_csv('assets/snar_experiments_external_1.csv')
prev_res

,tau,equiv_pldn,conc_dfnb,temperature,sty,e_factor,computation_t,experiment_t,strategy
0,0.65,2.2,0.46,75,7786.66,10.987,0,0.0123229,LHS
1,0.95,3,0.38,111,2887.74,20.6038,0,0.011729,LHS
2,1.25,4.6,0.14,57,1249.55,32.8315,0,0.00975466,LHS
3,1.85,3.8,0.3,39,1796.91,16.4831,0,0.0111432,LHS
4,1.55,1.4,0.22,93,1595.83,20.4113,0,0.00985384,LHS


Now, we can run TSEMO again and ask for new suggested conditions based on the first set of experiments.

In [26]:
# Get experimental suggestions from TSEMO
second_experiments = tsemo.suggest_experiments(5, prev_res=prev_res)

# Save TSEMO state
tsemo.save('assets/snar_tsemo_external_2.json')

# Save experiments to CSV
second_experiments.to_csv('assets/snar_experiments_external_2.csv')
second_experiments

,tau,equiv_pldn,conc_dfnb,temperature,sty,e_factor,strategy
418,0.503062,2.159430,0.333136,39.349317,8571.641870,20.263332,TSEMO2
417,1.942759,2.438533,0.218445,40.658336,1900.742195,20.262234,TSEMO2
340,0.937271,2.522983,0.455450,102.966221,3068.119187,20.300237,TSEMO2
896,1.273378,4.840805,0.212085,68.186840,1260.200774,21.023510,TSEMO2
506,0.915507,3.835045,0.486018,109.584684,3397.920469,20.275177,TSEMO2


We can repeat the the cycle of saving and loading the TSEMO paramters and next experiments until we are satisfied with the results.

## Running Closed Loop Experiments

Summit has special functionality for running closed-loop experiments.  The runner class takes a strategy and experiment. It then get experiment suggestions from the strategy and runs the experiments.

Note that in order to see the progress bar in jupyter, you'll need to install the jupyter or jupyterlab extension for ipywidgets. See [here](https://ipywidgets.readthedocs.io/en/latest/user_install.html#installing-the-jupyterlab-extension) for details.

In [5]:
# Import the necessary classes
from summit.benchmarks import Hartmann3D
from summit.strategies import SNOBFIT, NelderMead
from summit.utils.dataset import DataSet
from summit.run import Runner

In [36]:
# Establish the experiment. Here, we are using one of the single objective test functions
exp = Hartmann3D(maximize=True, constraints=False)

# Set up the strategy, passing in the experimental domain.
nm = NelderMead(exp.domain)

# Use the runner to run closed loop experiments
r = Runner(strategy=nm, experiment=exp, 
           max_iterations=5)
r.run()

We can also save and load the state of the runner to disk.

In [47]:
# Save everything to a file
r.save('assets/snobfit_hartmann.json')

# Load everything from a file
Runner.load('assets/snobfit_hartmann.json')